In [13]:
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize
import time

In [14]:
# Paste the url you obtained for your data
url='https://www.zalando.fr/api/catalog/articles?categories=promo-enfant&limit=84&offset=84&sort=sale'

In [15]:
#urls = ['https://api.github.com', 'https://www.zalando.fr/api/catalog/articles?categories=promo-enfant&limit=84&offset=84&sort=sale']

In [19]:
#response = requests.get('https://api.github.com')
#response

<Response [403]>

In [22]:
!ls

zalando-fr_api.ipynb  zalando.json


In [21]:
response = requests.get(url,
                        time.sleep(60), 
                        verify=False, 
                        timeout= 10,
                        port=4443)


KeyboardInterrupt: 

In [18]:
if response.status_code == 200:
    print('Success!')
elif response.status_code == 404:
    print('Not Found.')

In [ ]:
results = response.json()
results

In [ ]:
flattened_data = json_normalize(results)

flattened_data1 = json_normalize(flattened_data.articles[0])
flattened_data1

In [ ]:
# Get the total number of pages
total_pages=results['pagination']['page_count']

# Your code
df=pd.DataFrame()
for i in range(total_pages):
    k=84*i
    url=f'https://www.zalando.fr/api/catalog/articles?categories=promo-enfant&limit=84&offset={k}&sort=sale'
    response = requests.get(url)
    results = response.json()
    flattened_data = json_normalize(results)
    flattened_data1 = json_normalize(flattened_data.articles[0])
    flattened_data1=flattened_data1.set_index('sku')
    df = df.append(flattened_data1)

df

In [ ]:
df.brand_name.value_counts().index[0]

In [ ]:
#Our data is still text. Convert prices into numbers:
df['price.original']=df['price.original'].str.extract('(\d*,\d*)')
df['price.promotional']=df['price.promotional'].str.extract('(\d*,\d*)')

df['price.original'] = [x.replace(',', '.') for x in df['price.original']]
df['price.promotional'] = [x.replace(',', '.') for x in df['price.promotional']]


In [ ]:


df['discount_amount']=df['price.original'].astype(float)-df['price.promotional'].astype(float)
df1=df.copy()



In [ ]:
total_disc=df1.groupby(['brand_name']).sum().discount_amount

In [ ]:
total_disc.sort_values(ascending=False).index[0]

In [ ]:
total_disc[total_disc==0]